# Chapter 4
# Q-Learning and SARSA Applications


## Applying SARSA to Taxi-v2

    The Taxi Problem
    
    from "Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition"
    by Tom Dietterich
    
    Description:
    
    There are four designated locations in the grid world indicated by R(ed), G(reen), Y(ellow), and B(lue). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drives to the passenger's location, picks up the passenger, drives to the passenger's destination (another one of the four specified locations), and then drops off the passenger. Once the passenger is dropped off, the episode ends.
    
    Observations: 
    
    There are 500 discrete states since there are 25 taxi positions, 5 possible locations of the passenger (including the case when the passenger is in the taxi), and 4 destination locations. 
    
    Passenger locations:
    - 0: R(ed)
    - 1: G(reen)
    - 2: Y(ellow)
    - 3: B(lue)
    - 4: in taxi
    
    Destinations:
    - 0: R(ed)
    - 1: G(reen)
    - 2: Y(ellow)
    - 3: B(lue)
        
    Actions:
    There are 6 discrete deterministic actions:
    - 0: move south
    - 1: move north
    - 2: move east 
    - 3: move west 
    - 4: pickup passenger
    - 5: dropoff passenger
    
    Rewards: 
    There is a default per-step reward of -1,
    except for delivering the passenger, which is +20,
    or executing "pickup" and "drop-off" actions illegally, which is -10.
    
#### Attention: there are state agreggation, where different states become a single one. For example, the 

In [1]:
import gym
import numpy as np

import pandas as pd
import altair as alt
#alt.renderers.enable('png')

import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline


In [2]:
env = gym.make('Taxi-v3')
env.reset()

183

In [3]:
print(env.observation_space)
print(env.action_space)

Discrete(500)
Discrete(6)


In [4]:
state = env.reset()
steps = []
for step in range(3):
    
    #plt.imshow(env.render(mode='rgb_array'))
    #display.display(plt.gcf())
    #display.clear_output(wait=True)
    action = env.action_space.sample()
    step = env.step(action)
    next_state = env.step(action)[0]
    print('State: ',state)
    print('Action taken: ',action)
    print('Next state: ',step[0],'\nReward: ',step[1])
    state = next_state
    steps.append(env.render())
    
    print()
#env.close()

State:  464
Action taken:  4
Next state:  464 
Reward:  -10
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

State:  464
Action taken:  0
Next state:  464 
Reward:  -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

State:  464
Action taken:  3
Next state:  464 
Reward:  -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)



In [5]:
env.P[97]

{0: [(1.0, 197, -1, False)],
 1: [(1.0, 97, -1, False)],
 2: [(1.0, 97, -1, False)],
 3: [(1.0, 77, -1, False)],
 4: [(1.0, 97, -10, False)],
 5: [(1.0, 85, 20, True)]}

In [6]:
# First we will define the e-greedy function that will make the policy e-greedy

def e_greedy(Q, s , epsilon=0.1):
    
    if np.random.uniform(0,1) < epsilon:
        
        return np.random.randint(Q.shape[1])
    
    else:
        
        return np.argmax(Q[s])

In [7]:
# A function to test the policy midway the policy learning

def run_episodes(env, Q, num_episodes=100, to_print=False):
    
    total_rew = []
    
    state = env.reset()
    
    for _ in range(num_episodes):
        
        done = False
        
        game_rew = 0
        
        while not done:
            
            next_state, reward, done, _ = env.step(np.argmax(Q[state]))
            state = next_state
            game_rew += reward
            if done:
                state = env.reset()
                total_rew.append(game_rew)
        
    if to_print:
        
        print('Mean score: %.3f of %i games!'%(np.mean(total_rew), num_episodes))
    
    else:
        return np.mean(total_rew)

In [8]:
# Now we define the SARSA algorithm

def SARSA(env, alpha=0.01, num_episodes=10000, epsilon=0.3, gamma=0.95, eps_decay=0.00005):
    
    nA = env.action_space.n
    nS = env.observation_space.n
    test_rewards = []
    test_episodes = []
    Q = np.zeros((nS,nA))
    games_rewards = []
    
    for episode in range(num_episodes):
        
        state = env.reset()
        done = False
        total_rew = 0
        
        if epsilon > 0.01:
            
            epsilon -= eps_decay
        
        action = e_greedy(Q, state, epsilon)
        
        while not done:
            
            next_state, reward, done, _ = env.step(action)
            
            next_action = e_greedy(Q, next_state, epsilon)
            
            Q[state,action] = Q[state,action] + alpha * (reward + gamma * Q[next_state,next_action] - Q[state,action])
            
            state = next_state
            action = next_action
        
            total_rew += reward
            
            if done:
                games_rewards.append(total_rew)
                
        
        if (episode%300) == 0:
            
            test_rew = run_episodes(env, Q, 1000)
            print('Episode: {:5d} Eps: {:2.4f} Rew: {:2.4f}'.format(episode, epsilon, test_rew))
            test_episodes.append(episode)
            test_rewards.append(test_rew)
            
    
    return Q,test_rewards,test_episodes
    
    

In [9]:
Q, test_rewards, test_episodes = SARSA(env, alpha=0.1, num_episodes=5000, epsilon=0.4, gamma=0.95, eps_decay=0.001)

Episode:     0 Eps: 0.3990 Rew: -234.1730
Episode:   300 Eps: 0.0990 Rew: -213.5290
Episode:   600 Eps: 0.0100 Rew: -215.2560
Episode:   900 Eps: 0.0100 Rew: -129.6100
Episode:  1200 Eps: 0.0100 Rew: -114.6310
Episode:  1500 Eps: 0.0100 Rew: -86.5660
Episode:  1800 Eps: 0.0100 Rew: -42.5790
Episode:  2100 Eps: 0.0100 Rew: -19.3860
Episode:  2400 Eps: 0.0100 Rew: -19.9270
Episode:  2700 Eps: 0.0100 Rew: -2.7150
Episode:  3000 Eps: 0.0100 Rew: 4.0640
Episode:  3300 Eps: 0.0100 Rew: 7.6390
Episode:  3600 Eps: 0.0100 Rew: 7.8490
Episode:  3900 Eps: 0.0100 Rew: 7.8660
Episode:  4200 Eps: 0.0100 Rew: 7.8660
Episode:  4500 Eps: 0.0100 Rew: 7.8410
Episode:  4800 Eps: 0.0100 Rew: 7.9630


In [10]:
df_rewards_sarsa = pd.DataFrame()
df_rewards_sarsa['Episodes'] = test_episodes
df_rewards_sarsa['Mean_Rewards'] = test_rewards

In [11]:
sarsa_plot = alt.Chart(df_rewards_sarsa).mark_line().encode(alt.X('Episodes'),alt.Y('Mean_Rewards'))
sarsa_plot

alt.Chart(...)

## Applying Q-learning to Taxi-v2

In [12]:
# Define the Q-Learning algorithm

def Q_Learning(env, alpha=0.01, num_episodes=10000, epsilon=0.3, gamma=0.95, eps_decay=0.00005):
    
    nA = env.action_space.n
    nS = env.observation_space.n
    test_rewards = []
    test_episodes = []
    Q = np.zeros((nS,nA))
    games_rewards = []
    
    for episode in range(num_episodes):
        
        state = env.reset()
        done = False
        total_rew = 0
        
        if epsilon > 0.01:
            
            epsilon -= eps_decay
        
        while not done:
            
            action = e_greedy(Q, state, epsilon)
            
            next_state, reward, done, _ = env.step(action)
            
            Q[state,action] = Q[state,action] + alpha * (reward + gamma * np.max(Q[next_state]) - Q[state,action])
            
            state = next_state
        
            total_rew += reward
            
            if done:
                games_rewards.append(total_rew)
                
        
        if (episode%300) == 0:
            
            test_rew = run_episodes(env, Q, 1000)
            print('Episode: {:5d} Eps: {:2.4f} Rew: {:2.4f}'.format(episode, epsilon, test_rew))
            test_episodes.append(episode)
            test_rewards.append(test_rew)
            
    
    return Q,test_rewards,test_episodes
    
    

In [13]:
Q,test_rewards,test_episodes = Q_Learning(env,alpha=0.1,num_episodes=5000,epsilon=0.4,gamma=0.95,eps_decay=0.001)

Episode:     0 Eps: 0.3990 Rew: -241.2740
Episode:   300 Eps: 0.0990 Rew: -196.5670
Episode:   600 Eps: 0.0100 Rew: -170.5790
Episode:   900 Eps: 0.0100 Rew: -180.8450
Episode:  1200 Eps: 0.0100 Rew: -90.3760
Episode:  1500 Eps: 0.0100 Rew: -46.7960
Episode:  1800 Eps: 0.0100 Rew: -47.8230
Episode:  2100 Eps: 0.0100 Rew: -48.0230
Episode:  2400 Eps: 0.0100 Rew: -11.4690
Episode:  2700 Eps: 0.0100 Rew: -1.8460
Episode:  3000 Eps: 0.0100 Rew: 1.7570
Episode:  3300 Eps: 0.0100 Rew: 6.7940
Episode:  3600 Eps: 0.0100 Rew: 6.9580
Episode:  3900 Eps: 0.0100 Rew: 7.9460
Episode:  4200 Eps: 0.0100 Rew: 7.7680
Episode:  4500 Eps: 0.0100 Rew: 7.9890
Episode:  4800 Eps: 0.0100 Rew: 7.9060


In [14]:
df_rewards_ql = pd.DataFrame()
df_rewards_ql['Episodes'] = test_episodes
df_rewards_ql['Mean_Rewards'] = test_rewards

ql_plot = alt.Chart(df_rewards_ql).mark_line(color='red').encode(alt.X('Episodes'),alt.Y('Mean_Rewards'))
ql_plot

alt.Chart(...)

In [15]:
ql_plot+sarsa_plot


alt.LayerChart(...)